# i need to know how the customers are located geographically, both descriptive relying on maps and quantitatively apply the literature-established analyses


Generated by Auto-Analysis Web App

## Geospatial Data Preparation
This step performs essential data cleaning for geographic coordinates. It first identifies and reports the count of missing values in the 'latitude' and 'longitude' columns. Subsequently, rows containing any missing coordinates are removed from the dataset to ensure data integrity for spatial analysis. Finally, coordinate validation is performed by removing any records where 'latitude' falls outside the standard range of -90 to 90 degrees or 'longitude' outside -180 to 180 degrees. This ensures all remaining geographic data points are valid and can be accurately used in subsequent steps. The columns 'latitude' and 'longitude' are also explicitly converted to numeric types, coercing any errors to NaN, which are then dropped.

In [ ]:
import pandas as pdimport numpy as np# Check for missing values in 'latitude' and 'longitude'missing_lat_lon = df[['latitude', 'longitude']].isnull().sum()missing_report = f"Missing values in 'latitude': {missing_lat_lon['latitude']}\nMissing values in 'longitude': {missing_lat_lon['longitude']}"print(missing_report)# Handle missing geographic coordinates by removing rows with NaN in these columnsinitial_rows = len(df)df.dropna(subset=['latitude', 'longitude'], inplace=True)rows_after_na_drop = len(df)na_drop_report = f"Removed {initial_rows - rows_after_na_drop} rows with missing 'latitude' or 'longitude'."print(na_drop_report)# Ensure latitude and longitude are float types and handle potential coercion errorsdf['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')df.dropna(subset=['latitude', 'longitude'], inplace=True) # Drop rows that became NaN due to coercion# Validate coordinate ranges: latitude between -90 and 90, longitude between -180 and 180invalid_coords_mask = ~((df['latitude'].between(-90, 90)) & (df['longitude'].between(-180, 180)))invalid_coords_count = invalid_coords_mask.sum()df = df[~invalid_coords_mask]rows_after_validation = len(df)validation_report = f"Removed {rows_after_na_drop - rows_after_validation} rows with invalid coordinate ranges (-90 to 90 for latitude, -180 to 180 for longitude)."print(validation_report)print(f"Final dataset size after geospatial data preparation: {len(df)} rows.")# Store preprocessing report for potential display or logginggeospatial_prep_report = f"Geospatial Data Preparation Summary:\n{missing_report}\n{na_drop_report}\n{validation_report}\nFinal dataset size: {len(df)} rows."

## Descriptive Geographic Visualization
This step generates three interactive Folium maps to visually represent customer geographic distribution. First, a scatter plot (point map) marks each customer's location with a marker, aggregated using `MarkerCluster` for improved readability in dense areas, and includes `customer_id` in tooltips. Second, a heatmap is generated using `folium.plugins.HeatMap`, which uses color intensity to depict areas of higher customer concentration. Third, a gridded density map is created by aggregating customers into rectangular bins. Each bin's color intensity reflects the number of customers within that area, using a 'YlOrRd' colormap from `matplotlib` for clear density differentiation. All maps are centered at the average latitude and longitude of the customer data with an initial zoom level of 10 and utilize the 'OpenStreetMap' tile style.

In [ ]:
import foliumfrom folium.plugins import HeatMap, MarkerClusterimport mathimport matplotlib.pyplot as pltimport matplotlib.colorsimport matplotlib.cm# Parameters for Step 2map_base_tile_style = "OpenStreetMap"map_initial_zoom = 10map_initial_center_lat = 38.72map_initial_center_lon = -9.14# Ensure df has latitude and longitude after Step 1 and is not emptyif 'latitude' not in df.columns or 'longitude' not in df.columns or df.empty:    print("Warning: DataFrame does not contain 'latitude' and 'longitude' columns or is empty after preprocessing. Skipping geographic visualizations.")    m_points, m_heatmap, m_gridded = None, None, Noneelse:    # Update center coordinates based on actual data if available    map_initial_center_lat = df['latitude'].mean()    map_initial_center_lon = df['longitude'].mean()    # Sub-step 1: Interactive scatter plot (point map)    m_points = folium.Map(location=[map_initial_center_lat, map_initial_center_lon], zoom_start=map_initial_zoom, tiles=map_base_tile_style)    marker_cluster = MarkerCluster().add_to(m_points)    for idx, row in df.iterrows():        folium.Marker(            location=[row['latitude'], row['longitude']],            tooltip=f"Customer ID: {row['customer_id']}"        ).add_to(marker_cluster)    print("Generated interactive customer points map (m_points).")    # Sub-step 2: Heatmap of customer density    m_heatmap = folium.Map(location=[map_initial_center_lat, map_initial_center_lon], zoom_start=map_initial_zoom, tiles=map_base_tile_style)    HeatMap(df[['latitude', 'longitude']].values.tolist()).add_to(m_heatmap)    print("Generated customer density heatmap (m_heatmap).")    # Sub-step 3: Gridded map (simplified rectangular binning for density)    m_gridded = folium.Map(location=[map_initial_center_lat, map_initial_center_lon], zoom_start=map_initial_zoom, tiles=map_base_tile_style)    # Define grid boundaries and cell size    min_lat, max_lat = df['latitude'].min(), df['latitude'].max()    min_lon, max_lon = df['longitude'].min(), df['longitude'].max()    lat_range = max_lat - min_lat    lon_range = max_lon - min_lon    grid_bins_per_side = 20 # Example: 20x20 grid    grid_res_lat = lat_range / grid_bins_per_side if lat_range > 0 else 0.01    grid_res_lon = lon_range / grid_bins_per_side if lon_range > 0 else 0.01    grid_counts = {}    for idx, row in df.iterrows():        lat_bin = int(math.floor((row['latitude'] - min_lat) / grid_res_lat)) if grid_res_lat > 0 else 0        lon_bin = int(math.floor((row['longitude'] - min_lon) / grid_res_lon)) if grid_res_lon > 0 else 0                # Clamp bin indices to valid range (0 to grid_bins_per_side - 1)        lat_bin = max(0, min(lat_bin, grid_bins_per_side - 1))        lon_bin = max(0, min(lon_bin, grid_bins_per_side - 1))        key = (lat_bin, lon_bin)        grid_counts[key] = grid_counts.get(key, 0) + 1    max_count = max(grid_counts.values()) if grid_counts else 1    cmap = matplotlib.cm.YlOrRd # Yellow-Orange-Red colormap    # Add rectangles to map    for (lat_bin, lon_bin), count in grid_counts.items():        bottom_left_lat = min_lat + lat_bin * grid_res_lat        bottom_left_lon = min_lon + lon_bin * grid_res_lon        top_right_lat = bottom_left_lat + grid_res_lat        top_right_lon = bottom_left_lon + grid_res_lon        normalized_count = count / max_count if max_count > 0 else 0        color = cmap(normalized_count) # Get RGBA from colormap        color_hex = matplotlib.colors.rgb2hex(color) # Convert to hex string        folium.Rectangle(            bounds=[(bottom_left_lat, bottom_left_lon), (top_right_lat, top_right_lon)],            color=color_hex,            fill=True,            fill_color=color_hex,            fill_opacity=0.7,            tooltip=f"Customers: {count}"        ).add_to(m_gridded)    print("Generated gridded density map (m_gridded).")# To display the maps in a Jupyter environment, simply call the map variables:# m_points# m_heatmap# m_gridded

## Quantitative Geographic Clustering (DBSCAN)
This step applies the DBSCAN clustering algorithm to identify geographic customer clusters. Customer latitude and longitude coordinates are first converted to radians, which is necessary for accurate distance calculations using the Haversine metric. DBSCAN is then executed with an `eps` (maximum distance for neighborhood) of 0.5 kilometers (converted to radians) and `min_samples` (minimum points to form a dense region) of 10. The resulting cluster labels (or -1 for noise points) are added as a new 'cluster_id' column to the DataFrame. An interactive Folium map is generated, displaying each customer point colored according to its assigned cluster, with noise points colored gray. Finally, a report summarizes the total number of identified clusters, the count, and the proportion of customers in each cluster and labeled as noise.

In [ ]:
import foliumimport numpy as npfrom sklearn.cluster import DBSCANimport matplotlib.colorsimport matplotlib.cm# Parameters for Step 3dbscan_eps_km = 0.5 # kmdbscan_min_samples = 10# Ensure df has latitude and longitude after Step 1 and is not emptyif 'latitude' not in df.columns or 'longitude' not in df.columns or df.empty:    print("Warning: DataFrame does not contain 'latitude' and 'longitude' columns or is empty, cannot perform DBSCAN clustering.")    df['cluster_id'] = -2 # Assign a placeholder for unclustered if data is missing    m_clusters = None    cluster_report = "DBSCAN clustering skipped due to insufficient data."else:    # Sub-step 1: Convert latitude and longitude to radians for Haversine distance    coords = df[['latitude', 'longitude']].values    coords_radians = np.radians(coords)    # Sub-step 2: Apply DBSCAN algorithm    # eps_km needs to be converted to radians for the Haversine metric.    EARTH_RADIUS_KM = 6371.0 # Earth's radius in kilometers    dbscan_eps_radians = dbscan_eps_km / EARTH_RADIUS_KM    dbscan = DBSCAN(eps=dbscan_eps_radians, min_samples=dbscan_min_samples, algorithm='ball_tree', metric='haversine')    clusters = dbscan.fit_predict(coords_radians)    # Sub-step 3: Assign each customer to a discovered cluster    df['cluster_id'] = clusters    print("DBSCAN clustering applied and cluster_id assigned.")    # Sub-step 4: Visualize the identified clusters on an interactive map    map_initial_center_lat_dbscan = df['latitude'].mean()    map_initial_center_lon_dbscan = df['longitude'].mean()    # Reusing map_initial_zoom from Step 2 if it was defined, otherwise default to 10    map_initial_zoom = globals().get('map_initial_zoom', 10)     map_base_tile_style = globals().get('map_base_tile_style', "OpenStreetMap")    m_clusters = folium.Map(location=[map_initial_center_lat_dbscan, map_initial_center_lon_dbscan], zoom_start=map_initial_zoom, tiles=map_base_tile_style)    n_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)        # Use a diverse colormap for clusters, with gray for noise (-1)    cmap_clusters = matplotlib.cm.get_cmap('tab20', n_clusters)        cluster_colors = {}    unique_clusters = sorted(df['cluster_id'].unique())    cluster_idx = 0    for cluster_id in unique_clusters:        if cluster_id == -1:            cluster_colors[cluster_id] = 'gray' # Noise points        else:            cluster_colors[cluster_id] = matplotlib.colors.rgb2hex(cmap_clusters(cluster_idx))            cluster_idx += 1    for idx, row in df.iterrows():        cluster_id = row['cluster_id']        color = cluster_colors.get(cluster_id, 'black') # Default to black if cluster_id is unexpected        folium.CircleMarker(            location=[row['latitude'], row['longitude']],            radius=5,            color=color,            fill=True,            fill_color=color,            fill_opacity=0.7,            tooltip=f"Customer ID: {row['customer_id']}<br>Cluster: {cluster_id}"        ).add_to(m_clusters)    print("Generated interactive cluster map (m_clusters).")    # Sub-step 5: Calculate and report statistics    cluster_counts = df['cluster_id'].value_counts()    total_customers = len(df)    cluster_report = "\nDBSCAN Clustering Results:\n"    cluster_report += f"Total customers analyzed: {total_customers}\n"    cluster_report += f"Number of identified clusters: {n_clusters}\n"    for cluster_id, count in cluster_counts.items():        if cluster_id == -1:            cluster_report += f"  Noise points (-1): {count} customers ({count/total_customers:.2%})\n"        else:            cluster_report += f"  Cluster {cluster_id}: {count} customers ({count/total_customers:.2%})\n"    print(cluster_report)# To display the map in a Jupyter environment, simply call the map variable:# m_clusters

## Quantitative Geographic Density Analysis (KDE)
This step performs Kernel Density Estimation (KDE) to visualize customer density across the geographic area. It starts by projecting customer latitude and longitude into a local Cartesian coordinate system (kilometers from the mean center) to enable accurate Euclidean distance calculations required by `sklearn.neighbors.KernelDensity`. KDE is then applied with a 'gaussian' kernel and a bandwidth of 1.0 kilometers. A grid is established, based on `kde_grid_size_km` (0.1 km), covering the extent of customer locations, on which the density values are evaluated. An interactive Folium map is generated, displaying the KDE results as a raster image overlay, where color intensity represents customer density, created using `matplotlib.pyplot.contourf`. Finally, the density threshold corresponding to the top 10% of customer density is calculated and reported, highlighting primary customer territories visually represented by brighter colors on the map.

In [ ]:
import foliumimport numpy as npfrom sklearn.neighbors import KernelDensityimport matplotlib.pyplot as pltimport matplotlib.colorsimport matplotlib.cmimport ioimport base64# Parameters for Step 4kde_bandwidth_km = 1.0 # kmkde_grid_size_km = 0.1 # km# Ensure df has latitude and longitude after Step 1 and is not emptyif 'latitude' not in df.columns or 'longitude' not in df.columns or df.empty:    print("Warning: DataFrame does not contain 'latitude' and 'longitude' columns or is empty, cannot perform KDE.")    m_kde = None    kde_report = "KDE analysis skipped due to insufficient data."else:    # Project Lat/Lon to a local Cartesian plane (equirectangular approximation) in kilometers    mean_lat = df['latitude'].mean()    mean_lon = df['longitude'].mean()    mean_lat_rad = np.radians(mean_lat)    # Constants for degree to km conversion (approximate)    # 1 degree lat ~ 111.32 km    # 1 degree lon ~ 111.32 km * cos(latitude)        df_proj = df.copy()    df_proj['x_km'] = (df['longitude'] - mean_lon) * 111.32 * np.cos(mean_lat_rad)    df_proj['y_km'] = (df['latitude'] - mean_lat) * 111.32    # Sub-step 2: Apply Kernel Density Estimation (KDE)    kde = KernelDensity(kernel='gaussian', bandwidth=kde_bandwidth_km)    kde.fit(df_proj[['x_km', 'y_km']])    # Sub-step 1: Set up a grid for evaluation    min_x, max_x = df_proj['x_km'].min(), df_proj['x_km'].max()    min_y, max_y = df_proj['y_km'].min(), df_proj['y_km'].max()    # Extend grid slightly beyond min/max for smooth density edges    x_buffer_km = kde_bandwidth_km * 2    y_buffer_km = kde_bandwidth_km * 2    grid_min_x = min_x - x_buffer_km    grid_max_x = max_x + x_buffer_km    grid_min_y = min_y - y_buffer_km    grid_max_y = max_y + y_buffer_km    # Number of grid points based on grid_size_km    n_x_points = int(np.ceil((grid_max_x - grid_min_x) / kde_grid_size_km)) + 1    n_y_points = int(np.ceil((grid_max_y - grid_min_y) / kde_grid_size_km)) + 1        n_x_points = max(2, n_x_points) # Ensure at least 2 points    n_y_points = max(2, n_y_points)    x_grid_proj = np.linspace(grid_min_x, grid_max_x, n_x_points)    y_grid_proj = np.linspace(grid_min_y, grid_max_y, n_y_points)    X_GRID_PROJ, Y_GRID_PROJ = np.meshgrid(x_grid_proj, y_grid_proj)    xy_grid_proj = np.vstack([X_GRID_PROJ.ravel(), Y_GRID_PROJ.ravel()]).T    # Evaluate KDE on the grid    log_dens = kde.score_samples(xy_grid_proj)    kde_values = np.exp(log_dens) # Convert log-density back to density    density_map = kde_values.reshape(n_y_points, n_x_points)    print("KDE calculated on grid.")    # Sub-step 3: Generate an interactive contour/raster map    # Convert projected grid back to Lat/Lon for Folium bounds    lon_grid = (x_grid_proj / (111.32 * np.cos(mean_lat_rad))) + mean_lon    lat_grid = (y_grid_proj / 111.32) + mean_lat    # Map bounds for the image overlay    map_bounds = [        (lat_grid.min(), lon_grid.min()),        (lat_grid.max(), lon_grid.max())    ]    # Reusing map_initial_zoom from Step 2 if it was defined, otherwise default to 10    map_initial_zoom = globals().get('map_initial_zoom', 10)    map_base_tile_style = globals().get('map_base_tile_style', "OpenStreetMap")    m_kde = folium.Map(location=[mean_lat, mean_lon], zoom_start=map_initial_zoom, tiles=map_base_tile_style)    # Create a matplotlib contourf plot in memory and convert to base64 image    fig, ax = plt.subplots(figsize=(10, 10))    # Need to plot with actual lat/lon for contours, not projected. Regenerate meshgrid for lat/lon.    LON, LAT = np.meshgrid(lon_grid, lat_grid)    contour = ax.contourf(LON, LAT, density_map, levels=20, cmap='viridis', alpha=0.7)    plt.colorbar(contour, ax=ax, label='Customer Density')    ax.set_axis_off()    ax.set_aspect('equal', adjustable='box')    plt.tight_layout(pad=0)    buf = io.BytesIO()    plt.savefig(buf, format='png', bbox_inches='tight', pad_inches=0)    buf.seek(0)    img_base64 = base64.b64encode(buf.read()).decode('utf-8')    plt.close(fig)    folium.raster_layers.ImageOverlay(        image=f'data:image/png;base64,{img_base64}',        bounds=map_bounds,        opacity=0.7,        name='KDE Density Map'    ).add_to(m_kde)    folium.LayerControl().add_to(m_kde)    print("Generated interactive KDE density map (m_kde).")    # Sub-step 4: Identify and delineate the areas corresponding to the top X percentile of customer density    percentile_threshold = 90 # Top 10% density    if len(kde_values) > 0:        density_threshold = np.percentile(kde_values, percentile_threshold)    else:        density_threshold = 0.0 # Default if no density values    kde_report = "\nKDE Density Analysis Results:\n"    kde_report += f"KDE Bandwidth (km): {kde_bandwidth_km}\n"    kde_report += f"KDE Grid Size (km): {kde_grid_size_km}\n"    kde_report += f"Top {100 - percentile_threshold}% of customer density starts at a density value of approximately {density_threshold:.4f}.\n"    kde_report += f"These areas (shown in brighter colors on the map) represent primary customer territories."    print(kde_report)# To display the map in a Jupyter environment, simply call the map variable:# m_kde